# Test and use the created neural network




In [1]:
import numpy as np



from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread
from matplotlib import rcParams

import re
import cv2
from PIL import Image
import detectron2
import scipy.ndimage

import torch
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

import tensorflow as tf
from tensorflow import keras
import os

import cv2
import numpy as np
from matplotlib import pyplot as plt
#from scipy.misc import imsave
from scipy import ndimage
from scipy import misc
import scipy.misc
import scipy
import image_slicer
from image_slicer import join
from PIL import Image

2024-07-02 13:52:17.149166: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 13:52:20.677335: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


ModuleNotFoundError: No module named 'image_slicer'

In [ ]:

gpuAvailable = torch.cuda.is_available()

cfg = get_cfg()
cfg.OUTPUT_DIR = "./TrainDetectron2Model"

weightsPath = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
if not gpuAvailable:
    cfg.MODEL.DEVICE = 'cpu'


if not os.path.exists(weightsPath):
    print('cannot find weights at:', weightsPath, ', neural net detector cannot work.')
else:
    print('loading weights from', weightsPath)
    cfg.MODEL.WEIGHTS = weightsPath
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1   # set the testing threshold for this model
    cfg.TEST.DETECTIONS_PER_IMAGE = 6000
    predictor = DefaultPredictor(cfg)

In [ ]:
image_path = "../0_data/ParticlesFilter/Images/Image2.tif"


image = imread(image_path)

fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(image)

In [ ]:
print(type(image))

output = predictor(image)

print(output)

v = Visualizer(image[:, :, ::-1], # flip colors from RGB to BGR
                scale=0.8, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
v = v.draw_instance_predictions(output["instances"].to("cpu"))
    
    
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(v.get_image()[:, :, ::-1])

Cut image into pieces, annotate, reassemble

In [ ]:


img_path = '../0_data/ParticlesFilter/Images/Image2.tif'

full_image = imread(img_path)

#fig, ax = plt.subplots(figsize=(7, 7))
#ax.imshow(full_image)
#plt.show()



In [ ]:
# https://stackoverflow.com/questions/43565275/split-and-join-images-in-python
num_tiles = 20
tiles = image_slicer.slice(img_path, num_tiles)
print(tiles)

print("=================================")

for i, tile in enumerate(tiles):
    
    print(f"----- Image: {tile.filename} -----")
    
    # Convert PNG (float) to uint8 pixels
    image_tile = imread(tile.filename)
    image_tile *= 255
    image_tile = image_tile.astype('uint8')

    
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(image_tile)
    plt.show()
    
    
    current_output = predictor(image_tile)
    

    v = Visualizer(image_tile[:, :, ::-1], 
                    scale=0.8, 
                    instance_mode=ColorMode.IMAGE_BW   
    )
    v = v.draw_instance_predictions(current_output["instances"].to("cpu"))

    

    fig, ax = plt.subplots(figsize=(7, 7))
    ax.imshow(v.get_image()[:, :, ::-1])
    plt.show()

    
#image = join(tiles)

#image.save('./Image2_join.tif')

In [ ]:
image_png = imread("../0_data/ParticlesFilter/Images/Image2_01_01.png")
image_tif = imread("../0_data/ParticlesFilter/Images/Image2.tif")

print(type(image_png))
print(image_png.shape)
print(image_png[0])


print("--------------------")
print(type(image_tif))
print(image_tif.shape)
print(image_tif[0])



fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(image_png)
plt.show()


In [ ]:

output = predictor(image)

#print(output)

v = Visualizer(image[:, :, ::-1], # flip colors from RGB to BGR
                scale=0.8, 
                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
)
# Masks
for mask in output["instances"].pred_masks.to('cpu'):
    v.draw_soft_mask(mask)
v = v.get_output()
#out =  v.get_image()[:, :, ::-1]

  
    
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(v.get_image()[:, :, ::-1])
plt.show()